In [ ]:
import nltk
import pandas 
from sklearn.utils import shuffle
from sklearn.preprocessing import scale
import numpy as np
SENTENCE_LENGTH_ASSUMPTION=30

In [2]:
amazon=pandas.read_csv('sentiment labelled sentences\\amazon_cells_labelled.txt',sep='\t',header=None)
imdb=pandas.read_csv('sentiment labelled sentences\\imdb_labelled.txt',sep='\t',header=None)
yelp=pandas.read_csv('sentiment labelled sentences\\yelp_labelled.txt',sep='\t',header=None)
data=pandas.concat([amazon,imdb,yelp])
df = shuffle(data,random_state=1) #random_state=seed
print("Dataset count: ", len(data))
data = df.reset_index(drop=True)
data.rename(columns={0:'Sentence',1:'Label'}, inplace=True)
data.head(n=10)


Dataset count:  2748


,Sentence,Label
0,I mean this in a terrible way.,0
1,Perhaps I caught them on an off night judging ...,0
2,Later I found myself lost in the power of the ...,1
3,"But it is entertaining, nonetheless.",1
4,IT'S REALLY EASY.,1
5,"Speaking of the music, it is unbearably predic...",0
6,Not enough volume.,0
7,This place is amazing!,1
8,Battery is holding up well.,1
9,"Sorry, I will not be getting food from here an...",0


In [3]:
sentic_net=pandas.read_csv('senticnet\\senticnet4.txt',sep='\\s+',header=None,usecols=[0,2], names = ["Token", "Polarity"])
sentic_net = sentic_net[~sentic_net['Token'].str.contains('|'.join('_'),na=False)]
sentic_net = sentic_net.reset_index(drop=True)
print("Senticnet Vocab Size: ",len(sentic_net))
sentic_net=pandas.concat([sentic_net,pandas.DataFrame(data=scale(list(range(0,23682))),columns=['Vocab_Scale'])],axis=1)

sentic_net.head(n=10)

Senticnet Vocab Size:  23682


,Token,Polarity,Vocab_Scale
0,aaa,0.606,-1.731978
1,aah,-0.510,-1.731831
2,abandon,-0.560,-1.731685
3,abandonment,-0.650,-1.731539
4,abase,-0.580,-1.731393
5,abasement,-0.580,-1.731246
6,abash,-0.540,-1.731100
7,abashed,-0.520,-1.730954
8,abashment,-0.600,-1.730807
9,abate,-0.860,-1.730661


In [4]:
tagdict = nltk.load('help/tagsets/upenn_tagset.pickle')
pos_tags=dict.fromkeys(tagdict.keys())
pos_scale=scale(list(range(0,len(pos_tags))))
i=0
for k,v in pos_tags.items():
    pos_tags[k]=pos_scale[i]
    i+=1
    
pos_tags

{'$': 0.15399810070180361,
 "''": -1.4629819566671345,
 '(': -0.23099715105270544,
 ')': -0.15399810070180361,
 ',': 0.0,
 '--': -1.0009876545617236,
 '.': -0.076999050350901807,
 ':': -0.61599240280721446,
 'CC': 1.5399810070180362,
 'CD': 1.6169800573689381,
 'DT': -0.7699905035090181,
 'EX': 1.3089838559653308,
 'FW': 0.61599240280721446,
 'IN': 0.53899335245631264,
 'JJ': -1.1549857552635272,
 'JJR': 0.7699905035090181,
 'JJS': 0.84698955385991992,
 'LS': -1.6939791077198398,
 'MD': 1.0009876545617236,
 'NN': -0.84698955385991992,
 'NNP': 1.2319848056144289,
 'NNPS': -0.46199430210541087,
 'NNS': 1.3859829063162326,
 'PDT': 0.92398860421082174,
 'POS': 1.6939791077198398,
 'PRP': -0.69299145315811628,
 'PRP$': -0.38499525175450905,
 'RB': 0.23099715105270544,
 'RBR': 0.30799620140360723,
 'RBS': 0.38499525175450905,
 'RP': 0.69299145315811628,
 'SYM': 1.4629819566671345,
 'TO': -1.6169800573689381,
 'UH': -1.3089838559653308,
 'VB': 1.0779867049126253,
 'VBD': 0.46199430210541087,


In [137]:
#a=[]
#S="And his subtle connections between the three films are awesome.I have to mention this and it is a huge SPOILER, i loved the ending, how all the characters of the three films were the remaining survivors of the ferry disaster, with Valentine and the young judge together, and the old man watching it on her TV, solidifying his happiness over the suffering which he dealt with for those many years."
#pos_of_sent=nltk.pos_tag(nltk.word_tokenize(S))
#if len(pos_of_sent)>=29:
#    print(len(pos_of_sent))
#    a=create_feature_vector_of_sentence(pos_of_sent)



76


In [5]:
nltk.pos_tag(nltk.word_tokenize("I love football.")) ##Example output of pos tag parse

[('I', 'PRP'), ('love', 'VBP'), ('football', 'NN'), ('.', '.')]

In [163]:
def create_feature_vector_of_sentence(pos_of_sent):
    sentence_feature_vector=np.array([])
    word_idx=0
    for token, pos_tag in pos_of_sent:
        if word_idx>=SENTENCE_LENGTH_ASSUMPTION:
            return sentence_feature_vector.reshape(30,3)
        word_idx+=1
        
        if pos_tag not in pos_tags:
            print(pos_tag)
            word_idx-=1
            continue
        pos_polarity=pos_tags[pos_tag]
        
        token_lower=token.lower()
        lemma_token=nltk.WordNetLemmatizer().lemmatize(token_lower)        
        if token_lower is 'not':
            sentence_feature_vector=np.append(sentence_feature_vector,[0,-1,pos_polarity])
        elif lemma_token in sentic_net.Token.values:
            sentic_net_object=sentic_net.loc[sentic_net.Token == lemma_token]
            vocab_score=float(sentic_net_object.Vocab_Scale)
            pol_score= float(sentic_net_object.Polarity)
            sentence_feature_vector=np.append(sentence_feature_vector,[vocab_score,pol_score,pos_polarity])
        else:
            sentence_feature_vector=np.append(sentence_feature_vector,[0,0,pos_polarity])
        
    
    for _ in range(0,SENTENCE_LENGTH_ASSUMPTION-word_idx):
        sentence_feature_vector=np.append(sentence_feature_vector,[0,0,0])
    if len(sentence_feature_vector)==84:
        print (str(word_idx)  + " " + str(pos_of_sent))
    return sentence_feature_vector.reshape(30,3)

In [164]:
feature_vectors=np.array([]) #vocab_scale_polarity,sentic_net_polarity,pos_polarity
label_vector=np.array([])

for index, row in data.iterrows():
    pos_of_sent=nltk.pos_tag(nltk.word_tokenize(row.Sentence))
    feature_vectors=np.append(feature_vectors,create_feature_vector_of_sentence(pos_of_sent))
    label_vector=np.append(label_vector,row.Label)


#
#


In [172]:
f=np.array(feature_vectors)
f=f.reshape(len(data),30,3)
f.dump('features.npx')
f.shape

(2748, 30, 3)

In [174]:
l=np.array(label_vector,dtype=int)
l.dump('labels.npx')
l.shape

(2748,)